In [ ]:
!wget https://raw.githubusercontent.com/Amrtamer711/Shakespeare-Transformer/main/shakespeare_more.txt

--2023-11-27 22:13:03--  https://raw.githubusercontent.com/Amrtamer711/Shakespeare-Transformer/main/shakespeare_more.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5617411 (5.4M) [text/plain]
Saving to: ‘shakespeare_more.txt’

shakespeare_more.tx 100%[===================>]   5.36M  --.-KB/s    in 0.02s   

2023-11-27 22:13:04 (327 MB/s) - ‘shakespeare_more.txt’ saved [5617411/5617411]



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F # loading libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # mounting drive where files are stored

Mounted at /content/drive


In [ ]:
with open(r'shakespeare_more.txt', 'r', encoding='utf-8') as file:
    text = file.read() # reading the text from the dataset
unique_chars = sorted(list(set(text))) # creating a list of all the unique possible characters, the vocabulary
vocab_size = len(unique_chars) # checking the length of the vocabulary

In [ ]:
itos = {i:s for i, s in enumerate(unique_chars)} # creating a dictionary that converts integer index to corresponding token string character
stoi = {s:i for i, s in enumerate(unique_chars)} # creating a dictionary that converts token string character to corresponding integer index
encode = lambda x: [stoi[char] for char in x] # creating a lambda function that takes some charcaters/tokens and converts them into their corresponding integer indices
decode = lambda x: ''.join([itos[index] for index in x]) # creating a lambda function that takes some integer indices and converts them into their corresponding charcaters/tokens
data = torch.tensor(encode(text), dtype=torch.long) # converts the whole text into their corresponding token integer indices
n1 = int(len(data) * 0.8)
n2 = int(len(data) * 0.9)
data_train = data[:n1]
data_val = data[n1:n2]
data_test = data[n2:] # splitting the data set into a (train, val, test) split of (80, 10, 10)

In [ ]:
flat = 47104
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 96
context_size = 256
vector_length = 490
block_params = [(context_size, 1024, 2048), (64, 32), (3, 3), (1, 2), (2, 4), (0.2, 0.2)]
feed_params = [(flat, 5000, 5000, 5000), 0.2]
eval_interval = 500
eval_batch = 200
iterations = 30000
lr = 1e-3 # hyper-parameters for MLP model

In [ ]:
torch.cuda.is_available() # checking that GPU is usable

True

In [ ]:
class Convolution(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, pool_size, conv_stride, pool_stride, dropout):
        super().__init__()
        self.convolution = nn.Sequential(nn.Conv1d(in_channel, out_channel, kernel_size=kernel_size, stride=conv_stride),
                                        nn.BatchNorm1d(out_channel),
                                        nn.ReLU(),
                                        nn.MaxPool1d(kernel_size=pool_size, stride=pool_stride),
                                        nn.Dropout(dropout)) # creating convolution + pooling layer architecture for each individual layer
    def forward(self, x):
        self.out = self.convolution(x) # applying input through convolution layer
        return self.out

class Block(nn.Module):
    def __init__(self, channels, kernel_sizes, pool_sizes, conv_strides, pool_strides, dropouts):
        super().__init__()
        self.block = nn.Sequential(*[Convolution(channels[i], channels[i+1], kernel_sizes[i], pool_sizes[i], conv_strides[i], pool_strides[i], dropouts[i]) for i in range(len(channels)-1)]) # adding each convolution layer to model architecture
    def forward(self, x):
        self.out = self.block(x) # applying input through convolution + pooling layer
        return self.out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, fan_in, fan_out, dropout):
        super().__init__()
        self.linear = nn.Sequential(nn.Linear(fan_in, fan_out), nn.BatchNorm1d(fan_out), nn.ReLU(), nn.Dropout(dropout)) # creating linear layer architecture of each individual layer
    def forward(self, x):
        self.out = self.linear(x) # applying input through linear layer
        return self.out
class MLP(nn.Module):
    def __init__(self, shapes, dropout):
        super().__init__()
        self.mlp = nn.Sequential(*[FeedForward(shapes[i], shapes[i+1], dropout) for i in range(len(shapes)-1)]) # adding each linear layer to model architecture
    def forward(self, x):
        self.out = self.mlp(x)
        return self.out

In [ ]:
class CNN(nn.Module):
    def __init__(self, block_params, feed_params):
        super().__init__()
        self.char_embedding = nn.Embedding(vocab_size, vector_length) # creating character embedding look up table
        self.conv = Block(*block_params) # creating convolutional + pooling architecture
        self.mlp = MLP(*feed_params) # creating MLP/Feed Forward network architecture
        self.final = nn.Linear(feed_params[0][-1], vocab_size) # final linear layer (will not include batch norm, ReLU and dropout)
    def forward(self, x, targets=None):
        B, T = x.shape # getting tensor shape input
        embedding_matrix = self.char_embedding(x) # getting character embedding vectors of input text
        convoluted = self.conv(embedding_matrix) # applying input through convolutional part of CNN
        layer = convoluted.view(B, -1) # stretching second and third layer to be suitable for MLP
        forward = self.mlp(layer) # applying input through MLP/Feed Forward network
        logits = self.final(forward) # applying input through final linear layer
        if targets == None: # checking if in inference or training
            loss = None # if inference, no loss is needed
        else: # if training...
            loss = F.cross_entropy(logits, targets) # applying cross-entropy loss function
        return logits, loss
    def generate(self, idx, max_length): # function to generate text after training
        for _ in range(max_length): # ensuring output is of requested length
            idx_block = idx[:, -context_size: ]# take last context window length of tokens from complete context
            logits, loss = self(idx_block) # applying context window as input into MLP
            probs = F.softmax(logits, dim=-1) # apply softmax on logits to create probability distirbution
            char = torch.multinomial(probs, num_samples=1) # use weighted sampling to extract next token
            idx = torch.cat((idx, char), dim=1) # concatenate output character/token to previous text
        return idx

In [ ]:
@torch.no_grad()
def estimate_loss(): # function to estimate train and val loss while training
    out = {}
    model.eval() # setting model in evaluation mode
    for i in ['train', 'val']: # running over train and val datasets
        losses = torch.zeros(eval_batch) # creating lossses tensor of set evaluation batch length hyper-parameter
        for j in range(eval_batch): # running over losses tensor
            X_batch, Y_batch = get_batch(i) # get batch of inputs and expected outputs
            logits, loss = model(X_batch, Y_batch) # run inputs and expected outputs through model to get loss
            losses[j] = loss.item() # add value to losses tensor
        out[i] = losses.mean() # get mean of losses tensor and save it
    model.train() # put model back in train mode
    return out

@torch.no_grad()
def get_batch(mode): # function to collect batch of input/output pairs for training
    data = data_train if mode == 'train' else data_val # retrieve appropriate dataset
    batch = torch.randint(len(data) - context_size, (batch_size,)) # getting batches of random integers that will be used as dataset token integer indices of the starting token in the context window
    X_batch = torch.stack([data[i:i+context_size] for i in batch]) # stacking batches of context token indices to be used as input
    Y_batch = torch.stack([data[i+context_size] for i in batch]) # stacking batches of next character token index as output
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device) # moving batch to GPU
    return X_batch, Y_batch


@torch.no_grad()
def save_params(model, optimizer, scheduler):
  torch.save(model.state_dict(), r'/content/drive/MyDrive/ML_project/params.pt') # function to save parameters, optimizer and scheduler states
  torch.save(optimizer.state_dict(), r'/content/drive/MyDrive/ML_project/optimizer.pt') # to save parameters
  torch.save(scheduler.state_dict(), r'/content/drive/MyDrive/ML_project/scheduler.pt') # to save scheduler states

@torch.no_grad()
def test_model(model, data, batch_size): # function to test model on test set loss and accuracy
    cost = []
    accuracy = []
    for i in range(0, len(data) - context_size - batch_size, batch_size): # running model through whole test dataset
        X_batch = torch.stack([data[j:j+context_size] for j in range(i, i + batch_size)]) # getting test input batch
        Y_batch = torch.stack([data[j+context_size] for j in range(i, i + batch_size)]) # getting test expected output batch
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device) # moving batches to GPU
        logits, loss = model(X_batch, Y_batch) # running model through input/output batch
        cost.append(round(loss.item(), 4)) # get loss for batch and store it
        probs = F.softmax(logits, dim=-1) # apply softmax to get probability distirbution of next character token
        char = torch.multinomial(probs, num_samples=1).view(-1) # preform weighted sampling
        accuracy.append((len(char[char == Y_batch]) / 300) * 100) # check how common are expected and actual output equal
    test_cost = sum(cost) / len(cost) # calculate average test loss
    test_accuracy = sum(accuracy) / len(accuracy) # calculate average test accuracy
    return test_cost, test_accuracy

In [ ]:
model = CNN(block_params, feed_params) # creating CNN model
model.to(device) # moving model to GPU
optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # creating optimizer that is responsible for adjusting parameters
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95) # setting optimizer scheduler

In [ ]:
# model.load_state_dict(torch.load(r'/content/drive/MyDrive/ML_project/CNN_params.pt')) # importing model parameters
# optimizer.load_state_dict(torch.load(r'/content/drive/MyDrive/ML_project/CNN_optimizer.pt')) # importing model optimizer states
# scheduler.load_state_dict(torch.load(r'/content/drive/MyDrive/ML_project/CNN_scheduler.pt')) # importing model scheduler states

In [ ]:
model.eval() # putting model in evaluation mode
start = torch.zeros((1, context_size), device=device, dtype=torch.long) # creating full context character input start tokens to start generating
sample_index = model.generate(start, max_length=2000)[0].tolist() # generating output tokens inetger indices
sample = decode(sample_index) # converting integer indices of token into corresponding token string characters
print("Untrained sample is:\n", sample)
model.train() # putting model in train mode

In [2]:
for i in range(iterations): # running training over set number of iterations
    if i % eval_interval == 0 or i == iterations-1: # checking conditions on whether to stop temporarily in order to save parameters, estimate loss (and possibly apply scehduler)
        save_params(model, optimizer, scheduler) # saving parameters
        losses = estimate_loss() # estimate train and val loss
        print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if i != 0:
            scheduler.step() # performing scheduler step
    X_batch, Y_batch = get_batch('train') # collecting training batch
    logits, loss = model(X_batch, Y_batch) # running training batch through model
    optimizer.zero_grad(set_to_none=True) # setting gradients to 0 (to ensure gradients dont explode)
    loss.backward() # perform back prop
    optimizer.step()# perform parameter adjustment

step 0: train loss 4.6138, val loss 4.6138
step 500: train loss 2.5122, val loss 2.5149
step 1000: train loss 2.2791, val loss 2.2985
step 1500: train loss 2.1429, val loss 2.1556
step 2000: train loss 2.0710, val loss 2.0953
step 2500: train loss 1.9960, val loss 2.0333
step 3000: train loss 1.9646, val loss 2.0262
step 3500: train loss 1.8954, val loss 1.9659
step 4000: train loss 1.8847, val loss 1.9459
step 4500: train loss 1.8477, val loss 1.9270
step 5000: train loss 1.8377, val loss 1.8926
step 5500: train loss 1.7884, val loss 1.8815
step 6000: train loss 1.7677, val loss 1.8725
step 6500: train loss 1.7440, val loss 1.8311
step 7000: train loss 1.7377, val loss 1.8143
step 7500: train loss 1.7159, val loss 1.8063
step 8000: train loss 1.6860, val loss 1.8023
step 8500: train loss 1.6440, val loss 1.7880
step 9000: train loss 1.6457, val loss 1.7724
step 9500: train loss 1.6303, val loss 1.7691
step 10000: train loss 1.6345, val loss 1.7332
step 10500: train loss 1.5832, val lo

In [ ]:
model.eval() # putting model in evaluation mode
start = torch.zeros((1, context_size), device=device, dtype=torch.long) # creating full context character input start tokens to start generating
sample_index = model.generate(start, max_length=2000)[0].tolist() # generating output tokens inetger indices
sample = decode(sample_index) # converting integer indices of token into corresponding token string characters
print("Trained sample is:\n", sample)
model.train() # putting model in train mode

Trained sample is:
 change in copilions? Is you lame, nor goodly little fall our dismonths
If pleases and I’ll take the meanable prags.

DENALLOW.
The fox owt to Antiocan which is’t not.

MENENIUS.
Nay, I shall rey uncle, ’tis new if your silk.

KING,EDWARD.
Now to’t, were bestower it, Treason for an overhy, have peace, there despaid speak offended
Of you bear the word
Salisbury incertain of dost, and she tops usin. That high amoves that are and swift open!
The lost was w


In [ ]:
cost, accuracy = test_model(model, data_test, 300) # applying evaluation metrics on test set
print(f'Test loss is: {cost:.4f}\nTest accuracy: {accuracy:.2f}%')

Test loss is: 1.6526
Test accuracy: 41.98%
